# 差動駆動車のMPCでの軌道追従制御プログラムの実装

## pythonライブラリのインポート

In [2]:
#以下で使うライブラリのインポート
# 一般
import numpy as np
import cvxpy as cp
from typing import List, Tuple

# 描画関係
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrow
from PIL import Image
import io

## 差動駆動車のダイナミクス
差動駆動車のダイナミクスは、
$$\begin{aligned}
\dot{x} &= v \cos \theta \\
\dot{y} &= v \sin \theta \\
\dot{\theta} &= \omega
\end{aligned}$$
で、ここで、
$$\begin{aligned}
v &= \frac{v_R + v_L}{2} & \\
\omega &= \frac{v_R - v_L}{2l} \cos \alpha & (\alpha:\text{車輪の相対位置を決める角度},\; l\text{:車輪間の距離})
\end{aligned}
$$
である。ここで、$v_L,v_R$が独立変数で、制御量である。

### ダイナミクスを表すクラスの定義

In [3]:
class DiffCar:
	"""差動駆動車のクラス"""
	def __init__(self,wheel_distance,wheel_angle,x_init=0,y_init=0,theta_init=0):
		self.wheel_distance = wheel_distance
		self.wheel_angle = wheel_angle
		self.x = x_init
		self.y = y_init
		self.theta = theta_init
  
	def x_dot(self,x,y,theta,v_R,v_L):
		v = (v_R + v_L)/2
		x_dot = v*np.cos(theta)
		return x_dot

	def y_dot(self,x,y,theta,v_R,v_L):
		v = (v_R + v_L)/2
		y_dot = v*np.sin(theta)
		return y_dot

	def theta_dot(self,x,y,theta,v_R,v_L):
		v = (v_R + v_L)/2
		omega = (v_R - v_L)/(2*self.wheel_distance) *np.cos(self.wheel_angle)
		theta_dot = omega
		return theta_dot

	def dynamics(self,v_R,v_L) -> Tuple[float, float, float]:
		"""
		車輪の速度をv_R,v_Lとしたときの運動方程式を返す

		Parameters
		----------
		v_R : float
			 右車輪の速度
		v_L : float
			 左車輪の速度

		Returns
		-------
		Tuple[float, float, float]
			車の速度と角速度 
		"""
  
		v = (v_R + v_L)/2
		omega = (v_R - v_L)/self.wheel_distance *np.cos(self.wheel_angle)
		x_dot = v*np.cos(self.theta)
		y_dot = v*np.sin(self.theta)
		theta_dot = omega
		return x_dot,y_dot,theta_dot

	def runge_kutta_step(self,x, y, theta, dt, v_R, v_L):
		k1_x = dt * self.x_dot(x, y, theta, v_R, v_L)
		k1_y = dt * self.y_dot(x, y, theta, v_R, v_L)
		k1_theta = dt * self.theta_dot(x, y, theta, v_R, v_L)

		k2_x = dt * self.x_dot(x + 0.5 * k1_x, y + 0.5 * k1_y, theta + 0.5 * k1_theta, v_R, v_L)
		k2_y = dt * self.y_dot(x + 0.5 * k1_x, y + 0.5 * k1_y, theta + 0.5 * k1_theta, v_R, v_L)
		k2_theta = dt * self.theta_dot(x + 0.5 * k1_x, y + 0.5 * k1_y, theta + 0.5 * k1_theta, v_R, v_L)

		k3_x = dt * self.x_dot(x + 0.5 * k2_x, y + 0.5 * k2_y, theta + 0.5 * k2_theta, v_R, v_L)
		k3_y = dt * self.y_dot(x + 0.5 * k2_x, y + 0.5 * k2_y, theta + 0.5 * k2_theta, v_R, v_L)
		k3_theta = dt * self.theta_dot(x + 0.5 * k2_x, y + 0.5 * k2_y, theta + 0.5 * k2_theta, v_R, v_L)

		k4_x = dt * self.x_dot(x + k3_x, y + k3_y, theta + k3_theta, v_R, v_L)
		k4_y = dt * self.y_dot(x + k3_x, y + k3_y, theta + k3_theta, v_R, v_L)
		k4_theta = dt * self.theta_dot(x + k3_x, y + k3_y, theta + k3_theta, v_R, v_L)

		x_next = x + (k1_x + 2*k2_x + 2*k3_x + k4_x) / 6.0
		y_next = y + (k1_y + 2*k2_y + 2*k3_y + k4_y) / 6.0
		theta_next = theta + (k1_theta + 2*k2_theta + 2*k3_theta + k4_theta) / 6.0

		return x_next, y_next, theta_next

	def step(self,v_R,v_L,dt):
		"""
		車輪の速度をv_R,v_Lとして、dt時間だけ車を動かして、位置と姿勢を更新する

		Parameters
		----------
		v_R : float
				右車輪の速度
		v_L : float
				左車輪の速度
		dt : float
				時間
    
		Returns
		-------
		Tuple[float, float, float]
			車の位置と角度 
		"""
		#４次ルンゲクッタ法を用いて、位置と姿勢を更新
		self.x, self.y,self.theta = self.runge_kutta_step(self.x, self.y, self.theta, dt, v_R, v_L)

		return self.x, self.y, self.theta

	def reset(self,x_init=0,y_init=0,theta_init=0):
		self.x = x_init
		self.y = y_init
		self.theta = theta_init

### 車両描画用関数の定義

In [4]:
# 実際に動かしてその軌道をgifにする
def create_gif(x_list, y_list, theta_list, dt, fps):
    num_frames = int(len(x_list) // (fps * dt))

    pil_images = []
    x_list_range = max(x_list) - min(x_list)
    y_list_range = max(y_list) - min(y_list)
    xlim_min = min(x_list) - 0.5 * x_list_range
    xlim_max = max(x_list) + 0.5 * x_list_range
    ylim_min = min(y_list) - 0.5 * y_list_range
    ylim_max = max(y_list) + 0.5 * y_list_range

    for i in range(num_frames):

        fig, ax = plt.subplots()
        # グラフの縦横比を揃える
        ax.set_aspect('equal')
        # 赤線で軌跡を描画
        idx = int(i * (fps * dt))
        ax.plot(x_list[:idx], y_list[:idx], color='red')

        # 車の向きを矢印で描画
        dx = np.cos(theta_list[idx])*min(y_list_range,x_list_range) *0.1
        dy = np.sin(theta_list[idx])*min(y_list_range,x_list_range) *0.1
        arrow = FancyArrow(x_list[idx], y_list[idx], dx, dy, width=min(y_list_range,x_list_range)*0.02, color='blue')
        ax.add_patch(arrow)

        # XとYの軸を調整
        ax.set_xlim([xlim_min, xlim_max])
        ax.set_ylim([ylim_min, ylim_max])

        # イメージをバイト型で保存
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)

        # PIL Imageとしてロードしてリストに追加
        pil_image = Image.open(buf)
        pil_images.append(pil_image)
        plt.close(fig)

    # PIL Imageとして保存
    pil_images[0].save('output.gif',
                   save_all=True, append_images=pil_images[1:], optimize=False, duration=dt*1000, loop=0)

### 車両ダイナミクス・描画関数のテスト

In [5]:
#車のパラメータ
l = 0.02 #車輪間の距離[m]
alpha = np.pi/6 #車輪の角度[rad]

car = DiffCar(l,alpha,0.1,0.1,0)
dt = 0.1
total_steps =100
v_R = 0.03
v_L = 0.01

x_list = []
y_list = []
theta_list = []
for i in range(total_steps):
   x,y,theta = car.step(v_R,v_L,dt)
   x_list.append(x)
   y_list.append(y)
   theta_list.append(theta)
   
create_gif(x_list, y_list, theta_list, dt, fps=10)

## MPC用の線形ダイナミクスの導出
上式を$dt$間隔で離散化すると、
$$\begin{aligned}
x_{t+1} &= x_t + v_t \cos \theta_t dt \\
y_{t+1} &= y_t + v_t \sin \theta_t dt \\
\theta_{t+1} &= \theta_t + \omega_t dt
\end{aligned}$$
を得る。
いま、軌道追従制御を考え、理想となる軌道が与えられている状況を考える。
理想となる軌道を($x^{(\mathrm{ref})}_{t},y^{(\mathrm{ref})}_{t},\theta^{(\mathrm{ref})}_{t}$)として、上式を$x^{(\mathrm{ref})}_{t},y^{(\mathrm{ref})}_{t},\theta^{(\mathrm{ref})}_{t}$の周りでテイラー展開して線形化すると、

$$\begin{aligned}
x_{t+1} &= x_{t} + v_t \cos \theta^{(\mathrm{ref})}_{t} dt - v_t \sin \theta^{(\mathrm{ref})}_{t} (\theta_t-\theta^{(\mathrm{ref})}_{t}) dt + \mathcal{O}(dt^2) \\
y_{t+1} &= y_{t} + v_t \sin \theta^{(\mathrm{ref})}_{t} dt + v_t \cos \theta^{(\mathrm{ref})}_{t} (\theta_t-\theta^{(\mathrm{ref})}_{t}) dt + \mathcal{O}(dt^2) \\
\theta_{t+1} &= \theta_{t} + \omega_t dt
\end{aligned}$$
となる。さらに、理想となる軌道から、理想となる速度$v^{(\mathrm{ref})}_{t}$を近似的に求めることができるため、これを用いて制御量を$v_t = v^{(\mathrm{ref})}_{t} + u_t$とし、制御量が微小であると仮定する。すると、上式は近似的に
$$\begin{aligned}
x_{t+1} &= x_{t} + (v^{(\mathrm{ref})}_{t} + u_t) \cos \theta^{(\mathrm{ref})}_{t} dt - v^{(\mathrm{ref})}_{t} \sin \theta^{(\mathrm{ref})}_{t} (\theta_t-\theta^{(\mathrm{ref})}_{t}) dt + \mathcal{O}(dt^2) \\
y_{t+1} &= y_{t} + (v^{(\mathrm{ref})}_{t} + u_t) \sin \theta^{(\mathrm{ref})}_{t} dt + v^{(\mathrm{ref})}_{t} \cos \theta^{(\mathrm{ref})}_{t} (\theta_t-\theta^{(\mathrm{ref})}_{t}) dt + \mathcal{O}(dt^2) \\
\theta_{t+1} &= \theta_{t} + \omega_t dt
\end{aligned}$$
となる。
$$\begin{aligned}
\bm{x}_t = \begin{bmatrix}
x_{t} \\
y_{t} \\
\theta_{t}
\end{bmatrix},\;
\bm{u}_t = \begin{bmatrix}
u_{t} \\
\omega_{t}
\end{bmatrix}
\end{aligned}$$
とおいて、上式を整理して行列で表すと、
$$\begin{aligned}
\bm{x}_{t+1}
&=
\begin{bmatrix}
1 & 0 & -v^{(\mathrm{ref})}_{t} \sin \theta^{(\mathrm{ref})}_{t} dt \\
0 & 1 & v^{(\mathrm{ref})}_{t} \cos \theta^{(\mathrm{ref})}_{t} dt \\
0 & 0 & 1
\end{bmatrix}
\bm{x}_{t}
+
dt
\begin{bmatrix}
\cos \theta^{(\mathrm{ref})}_{t} & 0 \\
\sin \theta^{(\mathrm{ref})}_{t} & 0 \\
0 & 1
\end{bmatrix}
\bm{u}_{t}
+
v_t^{(\mathrm{ref})} dt
\begin{bmatrix}
\cos \theta^{(\mathrm{ref})}_{t} + \sin \theta^{(\mathrm{ref})}_{t} \theta^{(\mathrm{ref})}_{t} \\
\sin \theta^{(\mathrm{ref})}_{t} - \cos \theta^{(\mathrm{ref})}_{t} \theta^{(\mathrm{ref})}_{t} \\
0
\end{bmatrix}
\end{aligned}$$
となる。
本来の制御量である$v_{Rt},v_{Lt}$を、$v_{Rt} =v^{(\mathrm{ref})}_{t} + u_{Rt}, v_{Lt} = v^{(\mathrm{ref})}_{t} + u_{Lt}$、とおいて，制御量を$\bm{w}_t = \begin{bmatrix}u_{Rt} & u_{Lt}\end{bmatrix}^T$とする．
このとき，
$$\begin{aligned}
\bm{u}_t 
= \begin{bmatrix}
      u_{t} \\
      \omega_{t}
   \end{bmatrix}
   =
   \begin{bmatrix}
      \frac{v_{Rt} + v_{Lt}}{2}  - v_t^{(\mathrm{ref})} \\
      \frac{v_{Rt}-v_{Lt}}{2l} \cos \alpha 
   \end{bmatrix}
   =
   \begin{bmatrix}
      \frac{(v_{Rt} - v_t^{(\mathrm{ref})}) + (v_{Lt}- v_t^{(\mathrm{ref})})}{2}\\
      \frac{(v_{Rt} - v_t^{(\mathrm{ref})})-(v_{Lt}- v_t^{(\mathrm{ref})})}{2l} \cos \alpha 
   \end{bmatrix}
   =   
   \begin{bmatrix}
      \frac{u_{Rt} + u_{Lt}}{2}\\
      \frac{u_{Rt} - u_{Lt}}{2l} \cos \alpha 
   \end{bmatrix}
   =
   \begin{bmatrix}
      1/2 & 1/2 \\
      \cos \alpha/(2l) & -\cos \alpha/(2l) 
   \end{bmatrix}
   \begin{bmatrix}
      u_{Rt} \\
      u_{Lt}
   \end{bmatrix}
\end{aligned}$$
となるので，これを用いて線形ダイナミクスを表すと，
$$\begin{aligned}
\bm{x}_{t+1}
&=
\begin{bmatrix}
1 & 0 & -v^{(\mathrm{ref})}_{t} \sin \theta^{(\mathrm{ref})}_{t} dt \\
0 & 1 & v^{(\mathrm{ref})}_{t} \cos \theta^{(\mathrm{ref})}_{t} dt \\
0 & 0 & 1
\end{bmatrix}
\bm{x}_{t}
+
dt
\begin{bmatrix}
\cos \theta^{(\mathrm{ref})}_{t} & 0 \\
\sin \theta^{(\mathrm{ref})}_{t} & 0 \\
0 & 1
\end{bmatrix}
\begin{bmatrix}
   1/2 & 1/2 \\
   \cos \alpha/(2l) & -\cos \alpha/(2l) 
\end{bmatrix}
\begin{bmatrix}
   u_{Rt} \\
   u_{Lt}
\end{bmatrix}
+
v_t^{(\mathrm{ref})} dt
\begin{bmatrix}
\cos \theta^{(\mathrm{ref})}_{t} + \sin \theta^{(\mathrm{ref})}_{t} \theta^{(\mathrm{ref})}_{t} \\
\sin \theta^{(\mathrm{ref})}_{t} - \cos \theta^{(\mathrm{ref})}_{t} \theta^{(\mathrm{ref})}_{t} \\
0
\end{bmatrix} \\
&=\begin{bmatrix}
1 & 0 & -v^{(\mathrm{ref})}_{t} \sin \theta^{(\mathrm{ref})}_{t} dt \\
0 & 1 & v^{(\mathrm{ref})}_{t} \cos \theta^{(\mathrm{ref})}_{t} dt \\
0 & 0 & 1
\end{bmatrix}
\bm{x}_{t}
+ dt
\begin{bmatrix}
   \cos \theta^{(\mathrm{ref})}_{t}/2   & \cos \theta^{(\mathrm{ref})}_{t}/2  \\
   \sin \theta^{(\mathrm{ref})}_{t}/2   & \sin \theta^{(\mathrm{ref})}_{t}/2 \\
   \cos \alpha/(2l) & -\cos \alpha/(2l)
\end{bmatrix}
\bm{w}_t
+
v_t^{(\mathrm{ref})} dt
\begin{bmatrix}
\cos \theta^{(\mathrm{ref})}_{t} + \sin \theta^{(\mathrm{ref})}_{t} \theta^{(\mathrm{ref})}_{t} \\
\sin \theta^{(\mathrm{ref})}_{t} - \cos \theta^{(\mathrm{ref})}_{t} \theta^{(\mathrm{ref})}_{t} \\
0
\end{bmatrix}\\
&= A_t \bm{x}_t + B_t \bm{w}_t + C_t
\end{aligned}
$$


を得る．上式をMPC用の線形ダイナミクスとして用いる。

## MPCの実装
### 制約条件
制約条件に、入力$v_{Lt}, v_{Rt}$の最大絶対値$v_{\mathrm{max}}$を設定する。
$$
\left\{\begin{aligned}
-v_{\mathrm{max}} \leq v_{Lt} \leq v_{\mathrm{max}} \\
-v_{\mathrm{max}} \leq v_{Rt} \leq v_{\mathrm{max}}
\end{aligned}\right.
\Leftrightarrow
\left\{
\begin{aligned}
-v_{\mathrm{max}} - v_t^{(\mathrm{ref})} \leq u_{Lt} \leq v_{\mathrm{max}}- v_t^{(\mathrm{ref})} \\
-v_{\mathrm{max}}- v_t^{(\mathrm{ref})} \leq u_{Rt} \leq v_{\mathrm{max}}- v_t^{(\mathrm{ref})}
\end{aligned}\right.
$$



### 目的関数
目的関数に、軌道追従誤差を設定する。
$$\begin{aligned}
J = \sum_{t=0}^{T-1} \left( \left( x_{t} - x^{(\mathrm{ref})}_{t} \right)^2 + \left( y_{t} - y^{(\mathrm{ref})}_{t} \right)^2 + \left( \theta_{t} - \theta^{(\mathrm{ref})}_{t} \right)^2 \right)
\end{aligned}$$
ここで，$T$は予測ホライズンである。


### 実装

In [6]:
class ModelPredictiveController:
    def __init__(self, length: float, alpha: float, dt: float, v_max: float):
        self.length = length
        self.alpha = alpha
        self.dt = dt
        self.v_max = v_max        
    
    def calc_At(self, x_target: float, y_target: float,v_target: float,theta_target: float) -> np.ndarray:
        At = np.array(
            [
                [1, 0, -self.dt * np.sin(theta_target) * v_target],
                [0, 1, self.dt * np.cos(theta_target) * v_target],
                [0, 0, 1],
            ]
        )
        return At
    
    def calc_Bt(self, x_target: float, y_target: float,v_target: float, theta_target: float) -> np.ndarray:
        Bt = self.dt/2 * np.array(
            [
                [np.cos(theta_target), np.cos(theta_target)],
                [np.sin(theta_target), np.sin(theta_target)],
                [np.cos(self.alpha)/self.length, -np.cos(self.alpha)/self.length],
            ]
        )
        return Bt
    
    def calc_Ct(self, x_target: float, y_target: float,v_target: float, theta_target: float) -> np.ndarray:
        Ct = v_target * self.dt* np.array(
            [
                [np.cos(theta_target)+np.sin(theta_target)*theta_target],
                [np.sin(theta_target)-np.cos(theta_target)*theta_target],
                [0]
            ]
        )
        return Ct
    
    def calc_U(self,X_0,X_target:np.array)->np.ndarray:
        # 予測ホライズンの長さを取得
        horizon = X_target.shape[1] - 1
        
        # 変数
        X = cp.Variable((3, horizon+1)) # 状態変数(微少量)
        W = cp.Variable((2, horizon)) # 入力変数
        
        V_target = np.linalg.norm(np.diff(X_target[0:2,:]),axis=0,ord=2)/self.dt # 速度
        
        # 制約
        constraints = [X[:,0] == X_0] # 初期値
        for t in range(horizon):
            At = self.calc_At(X_target[0,t],X_target[1,t],V_target[t],X_target[2,t])
            Bt = self.calc_Bt(X_target[0,t],X_target[1,t],V_target[t],X_target[2,t])
            Ct = self.calc_Ct(X_target[0,t],X_target[1,t],V_target[t],X_target[2,t])
            # print("At.shape",At.shape)
            # print("Bt.shape",Bt.shape)
            # print("Ct.shape",Ct.shape)
            # print("right.shape",(At@X[:,t,None]+Bt@W[:,t,None]+Ct).shape)
            # print("V_target.shape",V_target.shape)
            constraints += [X[:,t+1,None] == At@X[:,t,None]+Bt@W[:,t,None]+Ct,
                            cp.abs(W[0,t] + V_target[t]) <= self.v_max,
                            cp.abs(W[1,t] + V_target[t]) <= self.v_max]
        
        # コスト関数
        cost = cp.sum_squares(X - X_target)
        # 最適化
        prob = cp.Problem(cp.Minimize(cost), constraints)
        prob.solve()
        
        solution = np.array(W.value)
        solution[0:2,:] = solution[0:2,:] + V_target[None,:] # 速度を足して実際の速度にする
        
        return solution

## 実験
### カージオイド軌道追従
カージオイド軌道を追従する実験を行う。カージオイド軌道は、
$$\begin{aligned}
x^{(\mathrm{ref})}_{t} &= a(2 \cos \omega t - \cos 2 \omega t)\\
y^{(\mathrm{ref})}_{t} &= a(2 \sin  \omega t - \sin 2 \omega t)\\
\theta^{(\mathrm{ref})}_{t} &= \tan^{-1} \left( \frac{\cos  \omega t - \cos 2 \omega t}{-\sin  \omega t + \sin 2 \omega t} \right)
\end{aligned}$$
である。(ただし、$a$はスケールを表すパラメータ，$\omega$は回転速度を表すパラメータである。)


今回は車輪間距離$l=0.02$ m、車輪の相対位置を決める角度$\alpha = \pi/6$ rad，タイヤの最大移動速度$v_{\mathrm{max}}=0.1$ m/s，回転速度$\omega = 2\pi/(10)$ rad/sec，スケール$a = 0.3$ m，予測ホライズン$T=10$とする。

### カージオイド軌道の生成関数

In [7]:
def generate_cardioid_trajectory(omega:float, scale:float,total_steps:int, dt:float)->np.ndarray:
      t = np.arange(0,total_steps*dt,dt)
      x = scale*(2*np.cos(omega*t)-np.cos(2*omega*t))
      y = scale*(2*np.sin(omega*t)-np.sin(2*omega*t))
      dx = -scale*(2*omega*np.sin(omega*t)-2*omega*np.sin(2*omega*t))
      dy = scale*(2*omega*np.cos(omega*t)-2*omega*np.cos(2*omega*t))
      theta = np.arctan2(dy,dx)
      
      #thetaが連続になるように調整
      theta = np.unwrap(theta)
      
      return np.array([x,y,theta])

### 車両・追従軌跡描画用関数の定義

In [8]:
# 実際に動かしてその軌道をgifにする
#軌道を描画
def create_gif_with_controller(X_real,X_target, dt, fps,filename="output_with_controller.gif"):
    num_frames = int(X_real.shape[1] // (fps * dt))

    pil_images = []
    X_max = np.max(X_target,axis=1)
    X_min = np.min(X_target,axis=1)
    ranges = X_max - X_min
    
    x_range = ranges[0] 
    y_range = ranges[1]
    lim_min = X_min - 0.5 * ranges
    lim_max = X_max + 0.5 * ranges
    
    xlim_min = lim_min[0]
    xlim_max = lim_max[0]
    ylim_min = lim_min[1]
    ylim_max = lim_max[1]

    for i in range(num_frames):

        fig, ax = plt.subplots()
        # グラフの縦横比を揃える
        ax.set_aspect('equal')
        # 赤線で理想軌跡を描画し
        idx = int(i * (fps * dt))
        ax.plot(X_target[0,:idx], X_target[1,:idx], color='red')
        # 青線で実際の軌跡を描画
        ax.plot(X_real[0,:idx], X_real[1,:idx], color='blue')

        # 車の向きを矢印で描画
        dx = np.cos(X_real[2,idx])*min(y_range,x_range) *0.1
        dy = np.sin(X_real[2,idx])*min(y_range,x_range) *0.1
        arrow = FancyArrow(X_real[0,idx], X_real[1,idx], dx, dy, width=min(y_range,x_range)*0.02, color='blue')
        ax.add_patch(arrow)
        # 理想の車の向きを赤で矢印で描画
        dx = np.cos(X_target[2,idx])*min(y_range,x_range) *0.1
        dy = np.sin(X_target[2,idx])*min(y_range,x_range) *0.1
        arrow = FancyArrow(X_target[0,idx], X_target[1,idx], dx, dy, width=min(y_range,x_range)*0.02, color='red')
        ax.add_patch(arrow)

        # XとYの軸を調整
        ax.set_xlim([xlim_min, xlim_max])
        ax.set_ylim([ylim_min, ylim_max])
        #　凡例を追加
        ax.legend(["target","real"])

        # イメージをバイト型で保存
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)

        # PIL Imageとしてロードしてリストに追加
        pil_image = Image.open(buf)
        pil_images.append(pil_image)
        plt.close(fig)

    # PIL Imageとして保存
    pil_images[0].save(filename,
                   save_all=True, append_images=pil_images[1:], optimize=False, duration=dt*1000, loop=0)
    

### 軌道と車両の生成

In [9]:
#目標軌道の生成
omega = 2*np.pi/10
scale = 0.1
total_steps = 100
dt = 0.1
X_target = generate_cardioid_trajectory(omega,scale,total_steps,dt)

#車のパラメータ
l = 0.02 #車輪間の距離[m]
alpha = np.pi/6 #車輪の角度[rad]
#車の実際の位置
# car = DiffCar(l,alpha,X_target[0,0],X_target[1,0],X_target[2,0]) #ノイズなし
# car = DiffCar(l*0.8,alpha*0.8,X_target[0,0],X_target[1,0],X_target[2,0]) #ノイズあり
# car = DiffCar(l*0.8,alpha*0.8,X_target[0,0]+0.1,X_target[1,0]+0.1,X_target[2,0]) #ノイズあり,位置ずれあり
car = DiffCar(l,alpha,X_target[0,0]+0.1,X_target[1,0]+0.1,X_target[2,0]) #位置ずれあり

### MPC制御器の生成と実行

In [10]:
#モデル予測制御器の生成
v_max = 0.5
horizon = 10
controller = ModelPredictiveController(l,alpha,dt,v_max)

#実際に動かしてみる
X_real = np.zeros((3,total_steps-horizon+1))
X_real[:,0] = np.array([car.x,car.y,car.theta])
for t in range(total_steps-horizon):
    #予測ホライズンの生成
    X_0 = np.array([car.x,car.y,car.theta])
    X_target_horizon = X_target[:,t:t+horizon+1]
    #最適化問題を解く
    U = controller.calc_U(X_0,X_target_horizon)
    #車を動かす
    car.step(U[0,0],U[1,0],dt)
    # 軌道を保存
    X_real[:,t+1] = np.array([car.x,car.y,car.theta])


#実行結果の可視化
create_gif_with_controller(X_real,X_target, dt, fps=10, filename='mpc_controller_noise8.gif')


# PID制御でなんとかしてみる
PID制御はSISO系以外には適用が難しいが、今回は意味的にPIDが適用できそうなので、これを検証してみる。
## 考察
今回のシステムは、2入力（$v_R, v_L$）3出力（$x,y,\theta$）である。
今回の目的は追従制御であるので、これを換言すれば位置・角度が一致すれば良い。
入力$v_R, v_L$は、$v_R = v + \omega l /\cos \alpha, v_L =  v - \omega l /\cos \alpha$と書けるので、
$v, \omega$を入力として制御を考える。このように入力を捉え直すことで、位置の問題と角度の問題を分離できる。
つまり、位置のズレを補正するPID制御を$v$に対して、角度のズレを補正するPID制御を$\omega$に対して行えば良い。
## 実装

In [ ]:
class PIDController:
   def __init__(self, Kp, Ki, Kd, dt):
      self.Kp = Kp
      self.Ki = Ki
      self.Kd = Kd
      self.dt = dt
      self.reset()

   def reset(self):
      self.P = 0
      self.I = 0
      self.D = 0
      self.prev_error = 0

   def step(self, error):
      self.P = error
      self.I += error * self.dt
      self.D = (error - self.prev_error) / self.dt
      self.prev_error = error
      return self.Kp * self.P + self.Ki * self.I + self.Kd * self.D
   
   
class DiffCarPIDController:
   def __init__(self, length: float, alpha: float, dt: float, v_max: float):
      self.length = length
      self.alpha = alpha
      self.v_max = v_max
      self.theta_pid = PIDController(10, 0.1, 1e-5, dt)
      self.pos_pid = PIDController(10, 0.1, 1e-5, dt)
      
   def reset(self):
      self.theta_pid.reset()
      self.pos_pid.reset()
      
   def step(self,X_0:np.ndarray, X_target:np.ndarray)->Tuple[float,float]:
      x, y, theta = X_0[0],X_0[1],X_0[2]
      x_target, y_target, theta_target = X_target[0],X_target[1],X_target[2]
      x_error = x_target - x
      y_error = y_target - y
      theta_error = theta_target - theta
      pos_error = np.sqrt(x_error ** 2 + y_error ** 2)
      omega = self.theta_pid.step(theta_error)
      v = self.pos_pid.step(pos_error)
      if v > self.v_max:
         v = self.v_max
      
      #v_R, v_Lを計算
      v_R = v + self.length * omega / np.cos(self.alpha)
      v_L = v - self.length * omega / np.cos(self.alpha)  
      
      return v_R, v_L

## 実行

In [ ]:
#制御器の生成
v_max = 0.5
pid_controller = DiffCarPIDController(l,alpha,dt,v_max)

#実際に動かしてみる
car.reset(X_target[0,0],X_target[1,0],X_target[2,0])
X_real = np.zeros((3,total_steps))
X_real[:,0] = np.array([car.x,car.y,car.theta])
for t in range(total_steps-1):
    #予測ホライズンの生成
    X_0 = np.array([car.x,car.y,car.theta])
    X_target_t = X_target[:,t]
    #最適化問題を解く
    v_R, v_L = pid_controller.step(X_0,X_target_t)
    #車を動かす
    car.step(v_R,v_L,dt)
    # 軌道を保存
    X_real[:,t+1] = np.array([car.x,car.y,car.theta])

In [ ]:
create_gif_with_controller(X_real,X_target, dt, fps=10,filename='pid_controller2.gif')